In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.4 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-03-20 02:25:09--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.22MB/s    in 0.8s    

2022-03-20 02:25:11 (1.22 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()


In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url ="https://data-finalproject.s3.amazonaws.com/Levels_Fyi_cleaned1.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get(""), sep=",", header=True, inferSchema=True)

In [5]:
df.show(5)

+--------------+---------+-----------------+-----------------------+------+--------------+----------------+-----+-------------+
|         title|  company|yearsofexperience|totalyearlycompensation|gender|Masters_Degree|Bachelors_Degree|state|         city|
+--------------+---------+-----------------+-----------------------+------+--------------+----------------+-----+-------------+
|Data Scientist| LinkedIn|              4.0|                 233000|  Male|             0|               0|   CA|San Francisco|
|Data Scientist|Microsoft|             11.0|                 218000|  Male|             0|               0|   WA|      Seattle|
|Data Scientist|     ebay|             10.0|                 180000|Female|             0|               0|   CA|     San Jose|
|Data Scientist|  Twitter|              4.0|                 500000|  Male|             0|               0|   CA|San Francisco|
|Data Scientist| Facebook|              8.0|                 370000|  Male|             0|              

In [9]:
df.dtypes

[('title', 'string'),
 ('company', 'string'),
 ('yearsofexperience', 'double'),
 ('totalyearlycompensation', 'int'),
 ('gender', 'string'),
 ('Masters_Degree', 'int'),
 ('Bachelors_Degree', 'int'),
 ('state', 'string'),
 ('city', 'string')]

In [10]:
salary_by_title_df = df.select(['title', 'totalyearlycompensation']).drop_duplicates()
salary_by_title_df.show(5)

+--------------+-----------------------+
|         title|totalyearlycompensation|
+--------------+-----------------------+
|Data Scientist|                 123000|
|Data Scientist|                 269000|
|Data Scientist|                 244000|
|Data Scientist|                  15000|
|Data Scientist|                 218000|
+--------------+-----------------------+
only showing top 5 rows



In [13]:
salary_by_title_and_gender_df = df.select(['title', 'totalyearlycompensation', 'gender']).drop_duplicates()
salary_by_title_and_gender_df.show(5)

+--------------+-----------------------+------+
|         title|totalyearlycompensation|gender|
+--------------+-----------------------+------+
|Data Scientist|                 182000|  Male|
|Data Scientist|                 123000|  null|
|Data Scientist|                 241000|  Male|
|Data Scientist|                 115000|  Male|
|Data Scientist|                 900000|  Male|
+--------------+-----------------------+------+
only showing top 5 rows



In [14]:
salary_by_experience_df = df.select(['title', 'yearsofexperience', 'totalyearlycompensation']).drop_duplicates()
salary_by_experience_df.show(5)

+--------------+-----------------+-----------------------+
|         title|yearsofexperience|totalyearlycompensation|
+--------------+-----------------+-----------------------+
|Data Scientist|              4.0|                 206000|
|Data Scientist|              5.0|                 306000|
|Data Scientist|             15.0|                 248000|
|Data Scientist|             13.0|                 187000|
|Data Scientist|              6.0|                 156000|
+--------------+-----------------+-----------------------+
only showing top 5 rows



In [15]:
salary_by_location_df = df.select(['title', 'state', 'city', 'totalyearlycompensation']).drop_duplicates()
salary_by_location_df.show(5)

+--------------+-----+-------------+-----------------------+
|         title|state|         city|totalyearlycompensation|
+--------------+-----+-------------+-----------------------+
|Data Scientist|   CA|    San Ramon|                 142000|
|Data Scientist|   CA|    Sunnyvale|                 130000|
|Data Scientist|   WA|      Redmond|                 320000|
|Data Scientist|   NY|     New York|                 284000|
|Data Scientist|   CA|San Francisco|                 285000|
+--------------+-----+-------------+-----------------------+
only showing top 5 rows



In [11]:
salary_by_education_df = df.select(['title', 'Masters_Degree', 'Bachelors_Degree', 'totalyearlycompensation']).drop_duplicates()
salary_by_education_df.show(5)

+--------------+--------------+----------------+-----------------------+
|         title|Masters_Degree|Bachelors_Degree|totalyearlycompensation|
+--------------+--------------+----------------+-----------------------+
|Data Scientist|             0|               0|                 235000|
|Data Scientist|             0|               0|                 325000|
|Data Scientist|             0|               0|                 141000|
|Data Scientist|             1|               0|                 108000|
|Data Scientist|             1|               0|                 390000|
+--------------+--------------+----------------+-----------------------+
only showing top 5 rows



In [16]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://final-project.cgernf5ijfvg.us-east-1.rds.amazonaws.com/postgres"
config = {"user":"postgres", 
          "password": "finalproject1", 
          "driver":"org.postgresql.Driver"}

In [18]:
salary_by_title_df.write.jdbc(url=jdbc_url, table="salary_by_title", mode=mode, properties=config)

In [19]:
salary_by_title_and_gender_df.write.jdbc(url=jdbc_url, table="salary_by_title_and_gender", mode=mode, properties=config)

In [20]:
salary_by_experience_df.write.jdbc(url=jdbc_url, table="salary_by_experience", mode=mode, properties=config)

In [21]:
salary_by_location_df.write.jdbc(url=jdbc_url, table="salary_by_location", mode=mode, properties=config)

In [22]:
salary_by_education_df.write.jdbc(url=jdbc_url, table="salary_by_education", mode=mode, properties=config)